<a href="https://colab.research.google.com/github/lexxai/goit_python_ds_hw_05/blob/main/goit_ds_hw_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LOAD DATASET

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

SAVED_DATAFRAME = Path("/content/data.feather")
URL = "https://drive.usercontent.google.com/download?id=1nzrtQpfaHL0OgJ_eXzA7VuEj7XotrSWO&export=download&authuser=0"
OUTPUT = Path("/content/homework.zip")
CSV_DATA_PATH  = Path("/content/data")

In [3]:
if SAVED_DATAFRAME.is_file():
  df = pd.read_feather(SAVED_DATAFRAME)
else:
  df = pd.DataFrame()

In [4]:
if not OUTPUT.is_file() and df.empty:
  !wget -O $OUTPUT $URL

if OUTPUT.is_file() and df.empty and not CSV_DATA_PATH.is_dir():
  !unzip -o $OUTPUT
  #!rm $OUTPUT

Показано результат, скорочений до останніх рядків (5000).
  inflating: data/running/running-1381.csv  
  inflating: data/running/running-1382.csv  
  inflating: data/running/running-1383.csv  
  inflating: data/running/running-1384.csv  
  inflating: data/running/running-1385.csv  
  inflating: data/running/running-1386.csv  
  inflating: data/running/running-1387.csv  
  inflating: data/running/running-1388.csv  
  inflating: data/running/running-1389.csv  
  inflating: data/running/running-139.csv  
  inflating: data/running/running-1390.csv  
  inflating: data/running/running-1391.csv  
  inflating: data/running/running-1392.csv  
  inflating: data/running/running-1393.csv  
  inflating: data/running/running-1394.csv  
  inflating: data/running/running-1395.csv  
  inflating: data/running/running-1396.csv  
  inflating: data/running/running-1397.csv  
  inflating: data/running/running-1398.csv  
  inflating: data/running/running-1399.csv  
  inflating: data/running/running-14.csv  


In [5]:
if df.empty:
  class_path = list(CSV_DATA_PATH.iterdir())
  class_list = list(d.name for d in class_path)
  print(class_list)

['stairs', 'idle', 'walking', 'running']


## BUILD DataFrame

In [34]:
def flatten_frame(frame):
        columns=[f"{col}_{i}" for i in range(frame.shape[0]) for col in frame.columns]
        return pd.DataFrame(frame.values.reshape(1, -1), columns=columns)

In [94]:
def add_stat_feature_frame(frame, rows):
  features = []
  features.append(pd.DataFrame([frame.iloc[:,0].mean()]*rows,columns=['X_mean']))
  features.append(pd.DataFrame([frame.iloc[:,1].mean()]*rows,columns=['Y_mean']))
  features.append(pd.DataFrame([frame.iloc[:,1].mean()]*rows,columns=['Z_mean']))
  result = pd.concat(features, axis=1)
  return result

In [95]:
df = pd.DataFrame()
if df.empty:
  FLATTEN = True
  STAT_FEATURE = True
  dfws = []
  limit_frames = None
  for class_id, work_class_path in enumerate(class_path):
    list_files = list(sorted(work_class_path.glob('*.csv'), key=lambda path: int(path.stem.rsplit("-", 1)[1])))
    print(f"Importing class '{work_class_path.name:7}' : {class_id}. Frames: {len(list_files)}")
    for i, filename in enumerate(list_files):
      # READ FRAME CSV FILE
      df_w: pd.DataFrame = pd.read_csv(filename)
      addon_features = [df_w]
      if FLATTEN:
        addon_features = [flatten_frame(df_w)]
      if STAT_FEATURE:
        addon_features.append(add_stat_feature_frame(df_w,addon_features[-1].shape[0]))
      df_w = pd.concat(addon_features, axis=1)
      df_w['class'] = class_id
      dfws.append(df_w)
      if limit_frames and (i > limit_frames):
        break
  df = pd.concat(dfws, axis=0, ignore_index=True)
# print(df.info())
print(df.shape)

Importing class 'stairs ' : 0. Frames: 165
Importing class 'idle   ' : 1. Frames: 1039
Importing class 'walking' : 2. Frames: 1850
Importing class 'running' : 3. Frames: 3408
(6462, 94)


In [96]:
df.sample(4)

,accelerometer_X_0,accelerometer_Y_0,accelerometer_Z_0,accelerometer_X_1,accelerometer_Y_1,accelerometer_Z_1,accelerometer_X_2,accelerometer_Y_2,accelerometer_Z_2,accelerometer_X_3,...,accelerometer_X_28,accelerometer_Y_28,accelerometer_Z_28,accelerometer_X_29,accelerometer_Y_29,accelerometer_Z_29,X_mean,Y_mean,Z_mean,class
659,0.009577,5.372589,8.169016,-0.392649,5.243302,7.814675,-0.225055,5.415684,7.642292,-0.478840,...,-0.124498,5.339070,8.188170,0.138864,5.133168,8.470686,-0.355619,5.426378,5.426378,1
5580,-1.843535,4.276045,6.703765,-1.747767,5.664682,-1.810016,0.301669,-18.555063,-0.938527,-1.474828,...,23.592464,23.003490,1.642422,-4.568136,8.930372,-3.260903,5.804822,6.770004,6.770004,3
551,0.282516,-0.019154,9.782708,0.272939,0.009577,9.768343,0.248997,-0.014365,9.777920,0.239420,...,0.263362,-0.009577,9.844957,0.201113,-0.014365,9.787497,0.255382,-0.010056,-0.010056,1
3494,17.975666,24.229320,14.470555,9.907207,18.033127,-7.881712,5.980716,-3.179500,-7.752425,-5.032612,...,19.115307,31.723173,0.636858,-4.381389,13.996503,7.149087,6.458599,9.085357,9.085357,3


In [97]:
if not df.empty and not SAVED_DATAFRAME.is_file():
  df.to_feather("data.feather")

In [98]:
for x in df['class'].unique():
  count = df['class'][df['class'] == x].count()
  prop = count / df.shape[0]
  print(f"class: {x}, rows: {count:7}, {class_list[x]:7}, prop: {prop:.4}" )

class: 0, rows:     165, stairs , prop: 0.02553
class: 1, rows:    1039, idle   , prop: 0.1608
class: 2, rows:    1850, walking, prop: 0.2863
class: 3, rows:    3408, running, prop: 0.5274


In [99]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
# X = np.array(df.iloc[:,:-1])
# y = np.array(df.iloc[:,-1])

In [100]:
X.shape

(6462, 93)

In [32]:
# X.corr()

### X SCALER

In [101]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


### train_test_split X_scaled

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score

SEED = 42

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=SEED, stratify=y)

## SVC

In [103]:
clf = SVC(probability=True)
clf.fit(X_train, y_train)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=0.9824651882413615


### Cross Validation

In [104]:
scores = cross_val_score(clf, X_train, y_train, cv=10)
scores.mean()

0.9827550841000996

### Count classes

In [16]:
y_test_pred = clf.decision_function(X_test)
y_test_pred.round(2)

array([[ 2.19, -0.27,  3.27,  0.79],
       [ 0.74, -0.22,  2.22,  3.26],
       [ 0.74, -0.24,  2.2 ,  3.27],
       ...,
       [ 0.75, -0.26,  2.22,  3.27],
       [ 0.75, -0.26,  2.23,  3.27],
       [ 0.74, -0.24,  2.2 ,  3.27]])

In [17]:
y_test_pred = clf.predict_proba(X_test)
y_test_pred.round(2)

array([[0.09, 0.  , 0.91, 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ],
       ...,
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ]])

In [18]:
y_test_pred = np.argmax(y_test_pred, axis=1)
y_test_pred

array([2, 3, 3, ..., 3, 3, 3])

In [19]:
y_test = clf.predict(X_test)

In [20]:
print(f"{y_test.shape=}")
y_pred_result = {}
for i in y_test_pred:
  y_pred_result[class_list[i]] = y_pred_result.get(class_list[i],0) + 1
print(f"{y_pred_result=}")

y_test_result = {}
for i in y_test:
  y_test_result[class_list[i]] = y_test_result.get(class_list[i],0) + 1
print(f"{y_test_result=}")

y_test.shape=(1939,)
y_pred_result={'walking': 571, 'running': 1023, 'idle': 312, 'stairs': 33}
y_test_result={'walking': 582, 'running': 1023, 'idle': 312, 'stairs': 22}


### PIPELINE SVC

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)

In [22]:
clf = make_pipeline(StandardScaler(), SVC(decision_function_shape='ovo'))
clf.fit(X, y)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=0.9876224858174316


In [23]:
clf = make_pipeline(StandardScaler(), SVC(decision_function_shape='ovr', kernel='linear'))
clf.fit(X, y)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=0.9819494584837545


In [24]:
clf = make_pipeline(StandardScaler(), SVC(decision_function_shape='ovr', kernel='poly', degree=4))
clf.fit(X, y)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=0.9742135121196493


In [25]:
numeric_features = X.columns
len(numeric_features)


90

## RandomForestClassifier

In [107]:
from sklearn.ensemble import RandomForestClassifier

clf = make_pipeline(StandardScaler(), RandomForestClassifier())
clf.fit(X, y)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=1.0


### Cross Validation

In [108]:
scores = cross_val_score(clf, X_train, y_train, cv=10)
scores.mean()

0.9770072671863096

## PIPE TEST

In [28]:
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numeric_features)
    ])

In [29]:
models = {
    'SVC': {
        'model': SVC(),
        'params': {}
    },
    'RandomForestClassifier': {
        'model': RandomForestClassifier(),
        'params': {}
    }
}

results = {}

In [30]:
for name, model_info in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model_info['model'])
    ])

    grid_search = GridSearchCV(pipeline,
                               model_info['params'],
                               cv=5,
                               scoring='recall',
                               )

    grid_search.fit(X_train, y_train)

    # y_pred = grid_search.predict(X_test)

    # classification_rep = classification_report(y_test, y_pred)
    # confusion_mat = confusion_matrix(y_test, y_pred)

    # results[name] = {
    #     'model': grid_search,
    #     'classification_report': classification_rep,
    #     'confusion_matrix': confusion_mat
    # }

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 2098, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1573, in precision_recall_fscore_support
    labels = _check_